# Example Notebook to showcasing how we interact with JTReaders

In [1]:
# First change dir to JTR parent
import os
os.chdir('..')

### Bookkeeping of all existing readers: `readers.py`

In [2]:
import jtr.jack.readers as readers

In [3]:
print("Existing models:\n%s" % ", ".join(readers.readers.keys()))

Existing models:
fastqa_reader, example_reader


### Create a reader

In [4]:
%%script bash
bash jtr/data/GloVe/download.sh

glove.6B.50d.txt already exists! Doing nothing!


In [5]:
from jtr.load.embeddings.embeddings import load_embeddings
from jtr.preprocess.vocab import Vocab

# we need a vocabulary (with embeddings for our fastqa_reader, but this is not always necessary)
embeddings = load_embeddings('jtr/data/GloVe/glove.6B.50d.txt', 'glove')
vocab = Vocab(emb=embeddings, init_from_embeddings=True)

# and a config
config = {"batch_size": 1, "repr_dim": 10, "repr_input_dim": embeddings.lookup.shape[1]}

[Loading GloVe]
  100k vectors processed...
  200k vectors processed...
  300k vectors processed...
  400k vectors processed...
[Loading GloVe DONE]


In [6]:
# create example reader
fastqa_reader = readers.readers["fastqa_reader"](vocab, config)

### Setting up a reader from training data 

In [7]:
from jtr.jack import load_labelled_data
train_data = load_labelled_data('data/SQuAD/snippet_jtr.json')
# all parameters are initialized after this call
fastqa_reader.setup_from_data(train_data)

### Saving the reader

In [8]:
fastqa_reader.store("/tmp/fastqa_reader")

In [9]:
%%sh
ls /tmp/fastqa_reader

checkpoint
model_module.data-00000-of-00001
model_module.index
model_module.meta
shared_resources


### Loading the reader

In [10]:
# we can simply load a setup reader
fastqa_reader.load("/tmp/fastqa_reader")

In [11]:
# or setup a new reader from file 
import tensorflow as tf
from jtr.preprocess.vocab import Vocab

# reset graph -> computation graph is gone
tf.reset_default_graph()
fastqa_reader = readers.readers["fastqa_reader"](None, None)
fastqa_reader.setup_from_file("/tmp/fastqa_reader")

### Applying the reader

In [12]:
from jtr.jack import load_labelled_data
train_data = load_labelled_data('data/SQuAD/snippet_jtr.json')

# take a list of inputs, e.g., from our training data
questions = [q for q, a in train_data]

In [13]:
fastqa_reader(questions)

[Answer(text='season. The American Football Conference', span=(121, 161), score=9.6192991e-05),
 Answer(text='league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily', span=(465, 568), score=0.00018284759),
 Answer(text='league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily', span=(465, 568), score=0.00019887803),
 Answer(text='Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily su